In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import socket
import numpy as np
from hashlib import md5
from Crypto.Cipher import AES
import os
import glob
import time
from Crypto import Random
from os import urandom
import tkinter as tk
from tkinter import *

In [3]:
HOST='localhost'
PORT=8000
# WIDTH = 960
# HEIGHT = 540
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((HOST, PORT))
s.listen(2)

In [4]:
def recv_all(sc, length):
    data = sc.recv(length)
    while True:
        miss_len_data = length - len(data)
        if miss_len_data==0:
            break
        data += sc.recv(miss_len_data)
    return data
def generateAESKey(key_string, len_key):
    '''Generate 128/192/256 bit key from any key'''
    key = bytes(key_string,"utf8")
    while len(key)<len_key:
        key = key + key
    return key[:len_key]

def make_folder(folder_name):
    path_f = os.path.abspath('')
    if os.path.isdir(path_f+"/"+folder_name) == False:
        os.mkdir(path_f+"/"+folder_name)
    list_folder = glob.glob(path_f+"/"+folder_name+"/*")
    max=0
    for i in list_folder:
        try:
            num = int(i.split("-")[1])
            if(max<num): max = num
        except: continue
    path = path_f+"/"+folder_name+"/data-"+str(max+1)
    os.mkdir(path)
    return path

# connect to client
def connect(s):
    print("Connecting...")
    try: 
        client, addr = s.accept()
        print('Connected by', addr)
        
    except:
        print("Run Client.py, Please")
    return client, addr


In [5]:
def derive_key_and_iv(key, salt, key_length, iv_length):
    d = d_i = b''
    while len(d) < key_length + iv_length:
        d_i = md5(d_i + key + salt).digest()
        d += d_i
    return d[:key_length], d[key_length:key_length+iv_length]

def encrypt(in_file, out_file, key, key_length=32):
    bs = AES.block_size #16 bytes
    salt = urandom(bs) #return a string of random bytes
    key, iv = derive_key_and_iv(key, salt, key_length, bs)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    out_file.write(salt)
    finished = False

    while not finished:
        chunk = in_file.read(1024 * bs) 
        if len(chunk) == 0 or len(chunk) % bs != 0:#final block/chunk is padded before encryption
            padding_length = (bs - len(chunk) % bs) or bs
            chunk += str.encode(padding_length * chr(padding_length))
            finished = True
        out_file.write(cipher.encrypt(chunk))
        
def decrypt(in_file, out_file, key, key_length=32):
    bs = AES.block_size
    salt = in_file.read(bs)
    key, iv = derive_key_and_iv(key, salt, key_length, bs)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    next_chunk = ''
    finished = False
    while not finished:
        chunk, next_chunk = next_chunk, cipher.decrypt(in_file.read(1024 * bs))
        if len(next_chunk) == 0:
            padding_length = chunk[-1]
            chunk = chunk[:-padding_length]
            finished = True 
        out_file.write(bytes(x for x in chunk)) 

In [6]:
def sendFile(client, filename, key, save_path):
    print("Waiting for Client... ")
    
    # encode message
    st_time = time.time()
    
    f_data_org = open(filename, 'rb')
    f_data_enc = open(save_path+"/Data-sent-to-client.txt", "wb")
    encrypt(f_data_org , f_data_enc , key)                             #encrypt
    f_data_org.close()
    f_data_enc.close()
    enc_time = time.time()
    print(" - Encoded AES-%d (%.3f seconds)" %(8*len('1'),enc_time-st_time))

    # Send message
    f_data_enc = f_data_enc = open(save_path+"/Data-sent-to-client.txt", "rb")
    data_enc = f_data_enc.read()
    f_data_enc.close()
    data_enc = bytes(data_enc)
#     print(len(data_enc))
    print(" - Sending data to client...")
    
    client.send(data_enc) # send encoded data 
    recv_all(client,4) # wait server response
    
    print("   Client has received data! " )
    print("Done!")

In [7]:
key = "Group_6"

key_AES = generateAESKey(key, 16) # create AES key (128bit)
print("Your AES key: ", key_AES)

Your AES key:  b'Group_6Group_6Gr'


In [8]:
client, addr = connect(s) 

Connecting...
Connected by ('127.0.0.1', 59791)


In [9]:
save_path = make_folder("Server_Folder")

In [10]:
window = Tk()
window.title('Server Window')
window.geometry('600x600')

label = Label(window, text = "Please enter your message !", font = ('Arial', 10))
label.place(x = 30, y = 20)
# entry = Entry(window, width = 20, font = ('Arial', 10))
# entry.place(x=30, y = 50)
textBox=Text(window , height=4, width=50)
textBox.place(x=30, y = 50)
def data_org(mess):
    f_data_org = open(save_path+"/Data-org.txt", "w")
    f_data_org.write(mess)
    f_data_org.close()
    
def button_command():
    mess = textBox.get("1.0","end-1c")
    data_org(mess)
    textBox.delete("1.0","end-1c")
    filename = save_path+"/Data-org.txt"
    print("Done write")
    sendFile(client, filename, key_AES, save_path)
        
    
button1 = Button(window, text = "Done", width = 10, height = 2, command = button_command)
button1.place(x = 40, y = 150)


#Lặp vô tận để hiển thị cửa sổ
window.mainloop()

Done write
Waiting for Client... 
 - Encoded AES-8 (0.009 seconds)
 - Sending data to client...
   Client has received data! 
Done!
Done write
Waiting for Client... 
 - Encoded AES-8 (0.002 seconds)
 - Sending data to client...


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\ADMIN\anaconda3\envs\Py38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_2304\1190475415.py", line 22, in button_command
    sendFile(client, filename, key_AES, save_path)
  File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_2304\3563119902.py", line 24, in sendFile
    recv_all(client,4) # wait server response
  File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_2304\3470271334.py", line 2, in recv_all
    data = sc.recv(length)
ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine


In [11]:
# filename = save_path+"/Data-org.txt"
# sendFile(client, filename, key_AES, save_path)